# MODNet with PBE band gaps as a feature

Band gap predictions using MODNet, experimental data as a training set and the PBE band gap values as a feature.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import os
import copy

from collections import defaultdict
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt 
from IPython.display import Markdown
from matminer.datasets import load_dataset, get_all_dataset_info
from pymatgen.core import Composition

from modnet.featurizers import MODFeaturizer
from modnet.featurizers.presets import DeBreuck2020Featurizer

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

In [5]:
md_joint = MODData.load('exp_pbe_joint')

2021-06-14 17:44:16,282 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f0050762dc0> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_structure

,structure
id,
mp-12699,"[[ 0.13616363 -2.60631371 1.50340165] Ca, [-1..."
mp-559459,"[[2.14049282 6.43464803 1.76199846] Cs, [-0.70..."
mp-21162,"[[0. 0. 0.] Sr, [0. 2.5259875 2.5259875..."
mp-1306,"[[3.259129 1.88166243 1.15971025] Th, [ 3.25..."
mp-15252,"[[0.95531657 1.042239 0.86229681] Cu, [2.803..."
...,...
mp-556541,"[[0. 6.49363948 5.82657217] In, [ 0. ..."
mp-2602,"[[3.023318 3.023318 3.023318] Nd, [0. 0. 0.] As]"
mp-30366,"[[4.51774558 1.37529211 4.32659015] Ca, [1.109..."


In [7]:
md_joint.df_featurized['Band gap PBE'] = md_joint.df_targets['pbe_gap']

In [8]:
md_joint.df_featurized

,ElementProperty|MagpieData minimum Number,ElementProperty|MagpieData maximum Number,ElementProperty|MagpieData range Number,ElementProperty|MagpieData mean Number,ElementProperty|MagpieData avg_dev Number,ElementProperty|MagpieData mode Number,ElementProperty|MagpieData minimum MendeleevNumber,ElementProperty|MagpieData maximum MendeleevNumber,ElementProperty|MagpieData range MendeleevNumber,ElementProperty|MagpieData mean MendeleevNumber,...,ChemEnvSiteFingerprint|GaussianSymmFuncstd_dev G2_80.0,ChemEnvSiteFingerprint|GaussianSymmFuncmean G4_0.005_1.0_1.0,ChemEnvSiteFingerprint|GaussianSymmFuncstd_dev G4_0.005_1.0_1.0,ChemEnvSiteFingerprint|GaussianSymmFuncmean G4_0.005_1.0_-1.0,ChemEnvSiteFingerprint|GaussianSymmFuncmean G4_0.005_4.0_1.0,ChemEnvSiteFingerprint|GaussianSymmFuncstd_dev G4_0.005_4.0_1.0,ChemEnvSiteFingerprint|GaussianSymmFuncmean G4_0.005_4.0_-1.0,ChemEnvSiteFingerprint|GaussianSymmFuncstd_dev G4_0.005_4.0_-1.0,ChemEnvSiteFingerprint|GaussianSymmFuncstd_dev G4_0.005_1.0_-1.0,Band gap PBE
mp-12699,20.0,79.0,59.0,53.714286,28.897959,79.0,7.0,66.0,59.0,40.714286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
mp-559459,16.0,55.0,39.0,34.000000,18.000000,16.0,5.0,88.0,83.0,64.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1704
mp-21162,38.0,82.0,44.0,71.000000,16.500000,82.0,8.0,81.0,73.0,62.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
mp-1306,13.0,90.0,77.0,32.250000,28.875000,13.0,16.0,73.0,57.0,58.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
mp-15252,16.0,32.0,16.0,23.000000,7.000000,16.0,64.0,88.0,24.0,78.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-556541,16.0,51.0,35.0,29.000000,16.000000,16.0,75.0,94.0,19.0,86.375000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5135
mp-2602,33.0,60.0,27.0,46.500000,13.500000,33.0,19.0,84.0,65.0,51.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
mp-30366,20.0,79.0,59.0,34.750000,22.125000,20.0,7.0,66.0,59.0,21.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
mp-3718,8.0,90.0,82.0,50.000000,28.000000,8.0,16.0,90.0,74.0,64.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2418


In [9]:
md_joint.df_targets = md_joint.df_targets.drop(columns=['pbe_gap', 'difference'])

In [10]:
import uncertainty_toolbox as uct

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
uncertainties = np.ones(5)
metricss = []
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_PBE-feature_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
      
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    ga = FitGenetic(train)
    model = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    metrics = uct.metrics.get_all_metrics(pred['exp_gap'].values, std['exp_gap'].values, true['exp_gap'].values)
    error = pred['exp_gap']-true['exp_gap']
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    std = std['exp_gap'].drop(std.index[((std['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    uncertainty = np.abs(std).mean()
    print('MAE =')
    print(mae)
    print('uncertainty =')
    print(uncertainty)
    maes[i] = mae
    uncertainties[i] = uncertainty
    metricss.append(metrics)

2021-06-14 17:44:16,490 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:16,505 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:16,516 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:16,531 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:16,541 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:16,557 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:16,567 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:16,583 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:16,593 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:16,610 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:44:17,092 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7effdae562b0> object, created with modnet version 0.1.9
2021-06-14 17

100%|██████████| 100/100 [11:14<00:00,  6.74s/it]

2021-06-14 17:55:33,235 - modnet - INFO - Loss per individual: ind 0: 0.987 	ind 1: 0.911 	ind 2: 0.911 	ind 3: 0.955 	ind 4: 0.987 	ind 5: 0.957 	ind 6: 0.922 	ind 7: 1.154 	ind 8: 0.905 	ind 9: 0.943 	ind 10: 0.919 	ind 11: 0.928 	ind 12: 0.976 	ind 13: 1.043 	ind 14: 0.965 	ind 15: 0.950 	ind 16: 0.994 	ind 17: 0.881 	ind 18: 0.893 	ind 19: 1.109 	


2021-06-14 17:55:34,760 - modnet - INFO - Generation number 1
2021-06-14 17:55:37,000 - modnet - INFO - Multiprocessing on 10 cores. Total of 128 cores available.


100%|██████████| 100/100 [11:34<00:00,  6.94s/it]

...

 (3/n) Calculating adversarial group calibration metrics
  [1/2] for mean absolute calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


 10%|█         | 1/10 [00:00<00:01,  7.41it/s]

  [2/2] for root mean squared calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


100%|██████████| 10/10 [00:01<00:00,  7.94it/s]

 (4/n) Calculating sharpness metrics
 (n/n) Calculating proper scoring rule metrics
**Finished Calculating All Metrics**


===================== Accuracy Metrics =====================
  MAE           0.346
  RMSE          0.699
  MDAE          0.024
  MARPD         136.681
  R2            0.839
  Correlation   0.920
=============== Average Calibration Metrics ================
  Root-mean-squared Calibration Error   0.201
  Mean-absolute Calibration Error       0.174
  Miscalibration Area                   0.176
========== Adversarial Group Calibration Metrics ===========
  Mean-absolute Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.227
     Group Size: 0.56 -- Calibration Error: 0.187
     Group Size: 1.00 -- Calibration Error: 0.174
  Root-mean-squared Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.255
     Group Size: 0.56 -- Calibration Error: 0.220
     Group Size: 1.00 -- Calibration Error: 0.201
==============

In [11]:
maes

array([0.36448238, 0.36444801, 0.30313387, 0.34086998, 0.34643761])

In [12]:
maes.mean()

0.3438743718191598

In [13]:
uncertainties

array([0.21845903, 0.19188686, 0.18078476, 0.17949469, 0.1999878 ])

In [14]:
uncertainties.mean()

0.19412262737751007

In [15]:
metricss

[{'accuracy': {'mae': 0.36448238013036294,
   'rmse': 0.761863198955022,
   'mdae': 0.08438744395971298,
   'marpd': 132.72021905337536,
   'r2': 0.7751361720408235,
   'corr': 0.8842640762160107},
  'avg_calibration': {'rms_cal': 0.09542256098834742,
   'ma_cal': 0.0844770283479961,
   'miscal_area': 0.08531969822292407},
  'adv_group_calibration': {'ma_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
    'adv_group_cali_mean': array([0.45786869, 0.14153854, 0.11149934, 0.10703349, 0.10323864,
           0.0982405 , 0.09345851, 0.09279935, 0.08887299, 0.08447703]),
    'adv_group_cali_stderr': array([0.03525213, 0.01862428, 0.01110936, 0.00725247, 0.00419611,
           0.00441212, 0.00532296, 0.00237604, 0.00162063, 0.        ])},
   'rms_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.66

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: 1%